In [25]:
import sys
import urllib.request
import requests
import datetime
import ijson
import pandas as pd
import xmltodict
import flatten_json
import os

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
sys.path.append('../../')

In [28]:
import irsparser as irs

# Main Parser

In [30]:
client = irs.Client(
    local_data_dir="../../data", ein_filename="eins",
    index_years=[2016, 2017, 2018], save_xml=False,
    parser="base")

Number of EINS: 2147
Gathered for Filings2016
Gathered for Filings2017
Gathered for Filings2018
Number of Jewish Orgs in Indices: 5153


In [31]:
client.parse_xmls(add_organization_info=True)

1000 records processed
Total Runtime 10.298592 seconds
2000 records processed
Total Runtime 19.633286 seconds
3000 records processed
Total Runtime 30.282277 seconds
4000 records processed
Total Runtime 41.093921 seconds


In [32]:
df = client.getFinalDF()

In [33]:
df.head()

,EIN,TaxPeriod,DLN,FormType,LastUpdated,ObjectId,OrganizationName,SubmittedOn,URL,Address,...,TotalLiabilitiesBOY,TotalLiabilitiesEOY,WorkingCapital,returnVersion,Deductibility,Foundation,Organization,NTEECode,NTEECommonCode,NTEECodeDescription
0,010267492,201511,93493067000346,990,2016-09-09T23:14:27,201640679349300034,PORTLAND CHEVRA KADISHA,2016-07-22,https://s3.amazonaws.com/irs-form-990/20164067...,471 Deering Avenue,...,6121.0,10578.0,0.000000,2014v6.0,Deductible,509(a)(2),Corporation,X20Z,"Religion-Related, Spiritual Development",Christian
1,010267492,201611,93493314027057,990,2018-01-18T23:53:03,201703149349302705,PORTLAND CHEVRA KADISHA,2018-01-05,https://s3.amazonaws.com/irs-form-990/20170314...,471 Deering Avenue,...,10578.0,30285.0,0.000000,2015v3.0,Deductible,509(a)(2),Corporation,X20Z,"Religion-Related, Spiritual Development",Christian
2,010267492,201711,93493030005318,990,2018-03-14T23:04:38,201810309349300531,PORTLAND CHEVRA KADISHA,2018-03-08,https://s3.amazonaws.com/irs-form-990/20181030...,471 Deering Avenue,...,30285.0,14393.0,0.000000,2016v3.0,Deductible,509(a)(2),Corporation,X20Z,"Religion-Related, Spiritual Development",Christian
3,010406624,201503,93493320093175,990,2016-03-21T17:23:53,201523209349309317,HOLOCAUST HUMAN RIGHTS CENTER OF MAINE,2016-02-04,https://s3.amazonaws.com/irs-form-990/20152320...,46 University Drive,...,4194.0,4302.0,9.008526,2014v5.0,Deductible,170(b)(1)(A)(vi),Corporation,Q700,"International, Foreign Affairs and National Se...",International Human Rights
4,010406624,201603,93493045023697,990,2017-09-13T21:58:58,201740459349302369,HOLOCAUST HUMAN RIGHTS CENTER OF MAINE,2017-07-27,https://s3.amazonaws.com/irs-form-990/20174045...,46 University Drive,...,4302.0,1326.0,8.405485,2015v3.0,Deductible,170(b)(1)(A)(vi),Corporation,Q700,"International, Foreign Affairs and National Se...",International Human Rights


In [34]:
df.columns

Index(['EIN', 'TaxPeriod', 'DLN', 'FormType', 'LastUpdated', 'ObjectId',
       'OrganizationName', 'SubmittedOn', 'URL', 'Address', 'BuildTs',
       'CYRevenuesLessExpenses', 'CYTotalExpenses', 'CYTotalRevenue', 'City',
       'CountryAbbr', 'GrantDesc', 'GrantMoneyTotal', 'LiabilitiesToAsset',
       'LoansFromOfficerBOY', 'LoansFromOfficerEOY', 'LoansToOfficerBOY',
       'LoansToOfficerEOY', 'Mission', 'NotFollowSFAS117',
       'OfficerCompensationPart7', 'OfficerCompensationPart9', 'OfficerList',
       'OfficerName', 'OfficerTitle', 'PYRevenuesLessExpenses',
       'PYTotalExpenses', 'PYTotalRevenue', 'PerparerFirmGrp', 'PreparerDate',
       'PreparerPersonName', 'ProgramExp', 'ProgramExpenses', 'ReturnTs',
       'ReturnTypeCd', 'ScheduleA', 'ScheduleI', 'ScheduleJ', 'ScheduleL',
       'ScheduleO', 'StateAbbr', 'SurplusMargin', 'TaxPeriodBeginDt',
       'TaxPeriodEndDt', 'TaxYr', 'TotalAssetsBOY', 'TotalAssetsEOY',
       'TotalEmployee', 'TotalExpenses', 'TotalLiabilitiesB

In [35]:
client.error_file#['url'].values

,error,url,version
0,'NoneType' object has no attribute 'get',https://s3.amazonaws.com/irs-form-990/20152323...,2012v3.0


In [36]:
df.to_csv(
    "../../output/forward_jewish_charities_990s_all.csv", header=True, index=False)

# Create DF_DASH - Dashboard

In [39]:
dash_cols = [
    # Org Info
    'EIN', 'URL', 'LastUpdated', 'OrganizationName', 'TaxPeriod', 'TaxPeriodBeginDt',
    'TaxPeriodEndDt', 'TaxYr', 'Address', 'City', 'StateAbbr', 'Mission', 'TotalEmployee', "ObjectId",
    'NTEECommonCode', 'Foundation',
    
    # People
    "OfficerName", 'OfficerCompensationPart9', "ScheduleJ",
    
    # Grants and Programs
    "GrantDesc", "GrantMoneyTotal", "ProgramExpenses",
    
    # Money
    "PYTotalRevenue", "CYTotalRevenue",
    "PYRevenuesLessExpenses", "CYRevenuesLessExpenses",
    'TotalAssetsBOY', 'TotalAssetsEOY', 
    'TotalLiabilitiesBOY', "TotalLiabilitiesEOY",
    'TotalExpenses', 'CYTotalExpenses', 'PYTotalExpenses',
    
    # Metrics
    "WorkingCapital", "LiabilitiesToAsset", "SurplusMargin", "ProgramExp",
    
    # Additional
    "ScheduleI", 'ScheduleO', "ScheduleA"]

In [40]:
df_dash = df.groupby(["EIN", "TaxYr"], as_index=False).last()[dash_cols]#.head()
df_dash.head(1)

,EIN,URL,LastUpdated,OrganizationName,TaxPeriod,TaxPeriodBeginDt,TaxPeriodEndDt,TaxYr,Address,City,...,TotalExpenses,CYTotalExpenses,PYTotalExpenses,WorkingCapital,LiabilitiesToAsset,SurplusMargin,ProgramExp,ScheduleI,ScheduleO,ScheduleA
0,010267492,https://s3.amazonaws.com/irs-form-990/20164067...,2016-09-09T23:14:27,PORTLAND CHEVRA KADISHA,201511,2014-12-01,2015-11-30,2014,471 Deering Avenue,Portland,...,188392.0,188392.0,201510.0,0.0,0.02373,0.078664,1.0,None,"{'@documentId': '00000004', '@softwareId': '14...","{'@documentId': '00000002', '@softwareId': '14..."


In [41]:
df_dash.to_csv("../../output/forward_jewish_charities_990s_dash.csv", header=True, index=False)

# Schedule I - Grants

In [42]:
for i,r in enumerate(df_dash.columns):
    print(i,r)

0 EIN
1 URL
2 LastUpdated
3 OrganizationName
4 TaxPeriod
5 TaxPeriodBeginDt
6 TaxPeriodEndDt
7 TaxYr
8 Address
9 City
10 StateAbbr
11 Mission
12 TotalEmployee
13 ObjectId
14 NTEECommonCode
15 Foundation
16 OfficerName
17 OfficerCompensationPart9
18 ScheduleJ
19 GrantDesc
20 GrantMoneyTotal
21 ProgramExpenses
22 PYTotalRevenue
23 CYTotalRevenue
24 PYRevenuesLessExpenses
25 CYRevenuesLessExpenses
26 TotalAssetsBOY
27 TotalAssetsEOY
28 TotalLiabilitiesBOY
29 TotalLiabilitiesEOY
30 TotalExpenses
31 CYTotalExpenses
32 PYTotalExpenses
33 WorkingCapital
34 LiabilitiesToAsset
35 SurplusMargin
36 ProgramExp
37 ScheduleI
38 ScheduleO
39 ScheduleA


In [50]:
grants = []
errors = 0
df_tmp = df_dash[[
    "EIN", "ObjectId", "OrganizationName", "TaxYr", "Address",
    "City", "StateAbbr", "ScheduleI"]].copy()
for row in df_tmp.itertuples():
    
    if row[8] is not None:
        tmp = {}
        tmp["EIN"] = row[1]
        tmp["ObjectId"] = row[2]
        tmp["OrganizationName"] = row[3]
        tmp["TaxYr"] = row[4]
        tmp["Address"] = row[5]
        tmp["City"] = row[6]
        tmp["StateAbbr"] = row[7]


        d = row[8]
        tbl = d.get("RecipientTable", False)
        if tbl:
            if isinstance(tbl, dict):
                # If its the only element in table, put it in a list to iterate over
                tmp2 = []
                tmp2.append(tbl)
                tbl = tmp2
                errors += 1
            for grant in tbl:
                tmp_grant = flatten_json.flatten(grant)
                
                tmp_grant.update(tmp)
                grants.append(tmp_grant)

In [53]:
df_grants = pd.DataFrame(grants)

In [54]:
df_grants.head().T

,0,1,2,3,4
Address,57 Ashmont Street,57 Ashmont Street,57 Ashmont Street,57 Ashmont Street,57 Ashmont Street
CashGrantAmt,69700,14465,17450,10817,5000
City,Portland,Portland,Portland,Portland,Portland
EIN,010530420,010530420,010530420,010530420,010530420
ForeignAddress_AddressLine1Txt,NaN,NaN,NaN,NaN,NaN
ForeignAddress_AddressLine2Txt,NaN,NaN,NaN,NaN,NaN
ForeignAddress_CityNm,NaN,NaN,NaN,NaN,NaN
ForeignAddress_CountryCd,NaN,NaN,NaN,NaN,NaN
ForeignAddress_ProvinceOrStateNm,NaN,NaN,NaN,NaN,NaN
IRCSectionDesc,NaN,NaN,NaN,NaN,NaN


In [57]:
grant_cols = [
    "EIN", "ObjectId", "OrganizationName", "TaxYr", "Address",
    "City", "StateAbbr", "RecipientEIN", "RecipientBusinessName_BusinessNameLine1Txt", "PurposeOfGrantTxt", "CashGrantAmt", 
    'NonCashAssistanceAmt', 'NonCashAssistanceDesc', "IRCSectionDesc", "USAddress_AddressLine1Txt", "USAddress_CityNm", "USAddress_StateAbbreviationCd",
    "ForeignAddress_AddressLine1Txt", "ForeignAddress_CountryCd"
]
df_grants[grant_cols].to_csv("../../output/forward_jewish_grants_to_orgs_all.csv", header=True, index=False)

In [23]:
# For Dashboard, set ScheduleI to True if exists else False
def scheduleIparser(x):
    if x is None:
        return False
    if x.get("RecipientTable", False):
        return True
    else:
        return False
    
df_dash["ScheduleI"] = df_dash["ScheduleI"].apply(scheduleIparser)

# Schedule A - OrgType Will be replaced with NTEE Codes eventually
## Could be other useful things in ScheduleA that will need to be explored

In [71]:
all_keys = []
for row in df_dash[["ScheduleA"]].itertuples():
    if row[1] is not None:
        all_keys = set(all_keys).union(list(row[1].keys()))

In [72]:
org_list = [org for org in list(all_keys) if "Ind" in org]

In [73]:
org_type = [
    "CollegeOrganizationInd", 'CommunityTrustInd', 'PrivateFoundation509Ind', 'PrivateFoundation170Ind', 
    'MajorityDirTrstSupportedOrgInd',
 'GovernmentalUnitInd',
 'PubliclySupportedOrg509a2Ind',
 'HospitalInd','SupportingOrganization509a3Ind',
 'SchoolInd',
 'PublicOrganization170Ind',
 'ChurchInd'
]
support_type = [
    'SupportingOrgType3FuncIntInd','First5Years170Ind',
 'ThirtyThrPctSuprtTestsCY509Ind','ThirtyThrPctSuprtTestsPY170Ind',
 'SupportingOrgType1Ind','SupportingOrgType2Ind','First5Years509Ind',
 'ThirtyThrPctSuprtTestsPY509Ind','TenPctFactsCrcmstncsTestPYInd',
 'SupportingOrgType3NonFuncInd', 'ThirtyThrPctSuprtTestsCY170Ind',
'TenPctFactsCrcmstncsTestCYInd'
]

irs_written = ['IRSWrittenDeterminationInd']

In [74]:
org_all = []
df_tmp = df_dash[["EIN", "ObjectId", "OrganizationName", "TaxYr", "StateAbbr", "ScheduleA"]].copy()
        
for row in df_tmp.itertuples():
    org_tmp = {}
    org_tmp["EIN"] = row[1]
    org_tmp["ObjectId"] = row[2]

    if row[6] is not None:
        for org in org_type:
            if row[6].get(org, False) :
                org_tmp["OrgType"] = org
        for org in support_type:
            if row[6].get(org, False) :
                org_tmp["OrgTypeSupport"] = org
        for org in irs_written:
            if row[6].get(org, False) :
                org_tmp["OrgTypeIrsDet"] = org
        org_all.append(org_tmp)
    else:
        org_all.append(org_tmp)

In [75]:
df_sched_a = pd.DataFrame(org_all)

In [76]:
df_sched_a.head()

,EIN,ObjectId,OrgType,OrgTypeIrsDet,OrgTypeSupport
0,010267492,201640679349300034,PubliclySupportedOrg509a2Ind,NaN,ThirtyThrPctSuprtTestsCY509Ind
1,010267492,201703149349302705,PubliclySupportedOrg509a2Ind,NaN,ThirtyThrPctSuprtTestsCY509Ind
2,010267492,201810309349300531,PubliclySupportedOrg509a2Ind,NaN,ThirtyThrPctSuprtTestsCY509Ind
3,010406624,201523209349309317,PublicOrganization170Ind,NaN,TenPctFactsCrcmstncsTestCYInd
4,010406624,201740459349302369,PublicOrganization170Ind,NaN,ThirtyThrPctSuprtTestsCY170Ind


## Remove Schedule A from df_dash

In [77]:
df_dash = pd.merge(df_dash, df_sched_a, on=["EIN", "ObjectId"])


In [78]:
# Set Schedule A to TrueFalse to indicate if it exists
def scheduleAparser(x):
    if x is None:
        return False
    else:
        return True
df_dash["ScheduleA"] = df_dash["ScheduleA"].apply(scheduleAparser)

# Schedule J

In [79]:
all_keys = []
for row in df_dash[['ScheduleJ']].itertuples():
    if row[1] is not None:
        all_keys = set(all_keys).union(list(row[1].keys()))

In [80]:
officers = []
#errors = 0
df_tmp = df_dash[["EIN", "ObjectId", "OrganizationName", "TaxYr", "StateAbbr", "ScheduleJ"]].copy()
        
for row in df_tmp.itertuples():
    if row[6] is not None:
        
        tmp = {}
        tmp["EIN"] = row[1]
        tmp["ObjectId"] = row[2]
        tmp["OrganizationName"] = row[3]
        tmp["TaxYr"] = row[4]
        tmp["StateAbbr"] = row[5]

        d = row[6]
        
        tmp["SeverancePaymentInd"] = d.get("SeverancePaymentInd", None)
        tmp["TravelForCompanionsInd"] = d.get("TravelForCompanionsInd", None)

        tbl = d.get("RltdOrgOfficerTrstKeyEmplGrp", False)
        if tbl:
            if isinstance(tbl, dict):
                # If its the only element in table, put it in a list to iterate over
                tmp2 = []
                tmp2.append(tbl)
                tbl = tmp2
                errors += 1
            for officer in tbl:
                tmp_officer = flatten_json.flatten(officer)

                tmp_officer.update(tmp)
                officers.append(tmp_officer)
    else:
        
        tmp = {}

In [81]:
df_officers = pd.DataFrame(officers)

In [82]:
df_officers.head()

,BaseCompensationFilingOrgAmt,BonusFilingOrganizationAmount,BonusRelatedOrganizationsAmt,BusinessName_BusinessNameLine1,BusinessName_BusinessNameLine1Txt,BusinessName_BusinessNameLine2,CompReportPrior990FilingOrgAmt,CompReportPrior990RltdOrgsAmt,CompensationBasedOnRltdOrgsAmt,DeferredCompRltdOrgsAmt,...,OtherCompensationFilingOrgAmt,OtherCompensationRltdOrgsAmt,PersonNm,SeverancePaymentInd,StateAbbr,TaxYr,TitleTxt,TotalCompensationFilingOrgAmt,TotalCompensationRltdOrgsAmt,TravelForCompanionsInd
0,230885,51000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,JEREMY ROTHSTEIN,false,CA,2014,COO,298487,NaN,None
1,148296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,MICHAEL DICKSON,false,CA,2014,Director - Israel,165616,NaN,None
2,253970,55000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ROZ ROTHSTEIN,false,CA,2014,CEO,330186,NaN,None
3,246127,0,0,NaN,NaN,NaN,0,0,0,0,...,0,0,ROZ ROTHSTEIN,0,CA,2016,CEO,265114,0,None
4,224856,0,0,NaN,NaN,NaN,0,0,0,0,...,0,0,JEREMY ROTHSTEIN,0,CA,2016,COO,232603,0,None


In [85]:
#df_officers.to_csv("output/forward_schedule_j.csv", header=True, index=False)

In [86]:
# Remove ScheduleJ
def scheduleJparser(x):
    if x is None:
        return False
    if x.get("RltdOrgOfficerTrstKeyEmplGrp", False):
        return True
    else:
        return False
df_dash["ScheduleJ"] = df_dash["ScheduleJ"].apply(scheduleJparser)

AttributeError: 'bool' object has no attribute 'get'


# Schedule O - Additional Information

In [87]:
def scheduleOParser(x):
    res_str = ""
    supplInfoDetail = x.get("SupplementalInformationDetail", {})
    if isinstance(supplInfoDetail, dict):
        # If its the only element in table, put it in a list to iterate over
        tmp2 = []
        tmp2.append(supplInfoDetail)
        supplInfoDetail = tmp2
    for detail in supplInfoDetail:
        res = detail.get("ExplanationTxt")
        res_str = " ".join([res_str, res])

    return res_str

In [88]:
df_dash["ScheduleO"] = df_dash["ScheduleO"].apply(scheduleOParser)

In [89]:
df_dash.columns

Index(['EIN', 'URL', 'LastUpdated', 'OrganizationName', 'TaxPeriod',
       'TaxPeriodBeginDt', 'TaxPeriodEndDt', 'TaxYr', 'StateAbbr', 'Mission',
       'TotalEmployee', 'ObjectId', 'NTEECommonCode', 'Foundation',
       'OfficerName', 'OfficerCompensationPart9', 'ScheduleJ', 'GrantDesc',
       'GrantMoneyTotal', 'ProgramExpenses', 'PYTotalRevenue',
       'CYTotalRevenue', 'PYRevenuesLessExpenses', 'CYRevenuesLessExpenses',
       'TotalAssetsBOY', 'TotalAssetsEOY', 'TotalLiabilitiesBOY',
       'TotalLiabilitiesEOY', 'TotalExpenses', 'CYTotalExpenses',
       'PYTotalExpenses', 'WorkingCapital', 'LiabilitiesToAsset',
       'SurplusMargin', 'ProgramExp', 'ScheduleI', 'ScheduleO', 'ScheduleA',
       'OrgType', 'OrgTypeIrsDet', 'OrgTypeSupport'],
      dtype='object')

In [92]:
df_dash.to_csv("output/forward_dashboard_final_v0.csv", header=True, index=False)